In [5]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models.phrases import Phrases, ENGLISH_CONNECTOR_WORDS

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
# %matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

Read in the texts

In [8]:
df = pd.read_csv('../data/aft.csv')
txt = df.text.values.tolist()
txt = [re.sub('\s+', ' ', sent) for sent in txt]
txt = [re.sub("\'", "", sent) for sent in txt]

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

words = list(sent_to_words(txt))

# print(words[:1])

In [9]:
bigram = gensim.models.Phrases(
  words, min_count = 10, threshold = 50, 
  connector_words = gensim.models.phrases.ENGLISH_CONNECTOR_WORDS
) # higher threshold fewer phrases.

trigram = gensim.models.Phrases(
  bigram[words], threshold=50,
  connector_words = gensim.models.phrases.ENGLISH_CONNECTOR_WORDS
)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# print(trigram_mod[bigram_mod[words[2]]])

# Define functions for stopwords, bigrams, trigrams and lemmatization

from gensim.parsing.preprocessing import STOPWORDS
# added_stopwords = STOPWORDS.union(set(['likes', 'play']))

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in STOPWORDS] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

words = make_bigrams(words)
words = make_trigrams(words)
words = remove_stopwords(words)

In [10]:
# Create Dictionary
id2word = corpora.Dictionary(words)
texts = words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# print(corpus[:1])
# [[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

lda_model = gensim.models.ldamodel.LdaModel(
  corpus=corpus, id2word=id2word, num_topics=20, 
  random_state=100, update_every=1, chunksize=100,
  passes=10, alpha='auto', per_word_topics=True
)

pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.084*"said" + 0.027*"went" + 0.023*"little" + 0.017*"came" + 0.015*"got" + '
  '0.014*"come" + 0.012*"home" + 0.010*"house" + 0.010*"oh" + 0.009*"going"'),
 (1,
  '0.195*"wolf" + 0.076*"goat" + 0.029*"journeyman" + 0.021*"george" + '
  '0.018*"mighty" + 0.017*"skin" + 0.014*"honey" + 0.012*"der" + 0.010*"bees" '
  '+ 0.010*"goats"'),
 (2,
  '0.202*"hare" + 0.079*"lion" + 0.078*"cage" + 0.018*"lions" + '
  '0.015*"animals" + 0.007*"forest" + 0.005*"buffaloes" + 0.005*"earth" + '
  '0.005*"range" + 0.004*"palm"'),
 (3,
  '0.116*"peter" + 0.065*"bridge" + 0.023*"millers" + 0.019*"boar" + '
  '0.018*"bran" + 0.017*"fought" + 0.010*"aye" + 0.009*"annoyed" + 0.004*"og" '
  '+ 0.001*"elks"'),
 (4,
  '0.044*"heels" + 0.041*"raja" + 0.027*"tank" + 0.022*"traveler" + '
  '0.021*"rupees" + 0.020*"obtain" + 0.013*"pitcher" + 0.013*"swans" + '
  '0.012*"flat" + 0.012*"plowing"'),
 (5,
  '0.051*"thee" + 0.033*"thy" + 0.030*"thou" + 0.025*"wild" + 0.023*"juan" + '
  '0.021*"ye" + 0.017*"snak

In [11]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

C:\Users\joshh\Documents\GitHub\trilogy\my_env\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.401741  0.055154       1        1  33.774504
0      0.412452  0.033018       2        1  23.194745
11     0.353156  0.021912       3        1  21.942228
15     0.384321  0.028492       4        1  14.556369
5      0.061509 -0.385729       5        1   1.944522
18    -0.055601 -0.012809       6        1   1.048823
7     -0.060570 -0.022098       7        1   0.778370
10    -0.091321  0.010137       8        1   0.747875
1     -0.104749  0.018196       9        1   0.329259
14    -0.114600  0.020568      10        1   0.327224
2     -0.108472  0.025908      11        1   0.258428
19    -0.113807  0.026523      12        1   0.246199
4     -0.119973  0.023987      13        1   0.206733
8     -0.118023  0.026006      14        1   0.179879
3     -0.120968  0.024405      15        1   0.141260
16    -0.122021  0.023044      16        1   0.100577
12    -0.121424  0.021861      17        1   0.092683
17    -0.121363  0.021666      18        1   0.067284
6     -0.120372  0.020208      19        1   0.032330
13    -0.119914  0.019550      20        1   0.030707, topic_info=            Term          Freq         Total Category  logprob  loglift
168         said  11505.000000  11505.000000  Default  30.0000  30.0000
2258        king   4073.000000   4073.000000  Default  29.0000  29.0000
1895         fox    774.000000    774.000000  Default  28.0000  28.0000
360       prince   1662.000000   1662.000000  Default  27.0000  27.0000
2071         cat    794.000000    794.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
24460      ferko      0.005452      0.683992  Topic20 -10.1986   3.2565
16638   huntsman      0.005452      0.683997  Topic20 -10.1986   3.2565
8789   shoemaker      0.005452      0.684027  Topic20 -10.1986   3.2565
7088      sexton      0.005452      0.683985  Topic20 -10.1986   3.2565
12460     jackal      0.005452      0.684076  Topic20 -10.1986   3.2564

[923 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
2275      3  0.999018      according
1937      3  0.994867        account
5433     11  0.569812  acquaintances
1784      8  0.959001             ad
3482      3  0.995757            age
...     ...       ...            ...
5510      4  0.993529   your_majesty
1655      3  0.421453          youth
1655      4  0.473290          youth
1655      5  0.103673          youth
6606     10  0.863035         zurich

[1079 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 1, 12, 16, 6, 19, 8, 11, 2, 15, 3, 20, 5, 9, 4, 17, 13, 18, 7, 14])